In [1]:
import torch
import numpy as np

In [2]:
torch.manual_seed(0)

In [7]:
#declare dimension
input_size = 4
hidden_size = 2

In [8]:
#input dimension(batch, sequence_length, input_size)
h = [1,0,0,0]
e = [0,1,0,0]
l = [0,0,1,0]
o = [0,0,0,1]

input_data_np = np.array([[h,e,l,l,o],
                          [e,o,l,l,l],
                          [l,l,e,e,l]], dtype = np.float32)
print(input_data_np.shape)

# Tensor로 변형
input_data = torch.Tensor(input_data_np)

(3, 5, 4)


In [9]:
# model
rnn = torch.nn.RNN(input_size, hidden_size)

In [12]:
#outputs
outputs, _status = rnn(input_data)
print(outputs)
print(outputs.size())

tensor([[[-0.7497, -0.6135],
         [-0.5282, -0.2473],
         [-0.9136, -0.4269],
         [-0.9136, -0.4269],
         [-0.9028,  0.1180]],

        [[-0.5753, -0.0070],
         [-0.9052,  0.2597],
         [-0.9173, -0.1989],
         [-0.9173, -0.1989],
         [-0.8996, -0.2725]],

        [[-0.9077, -0.3205],
         [-0.8944, -0.2902],
         [-0.5134, -0.0288],
         [-0.5134, -0.0288],
         [-0.9127, -0.2222]]], grad_fn=<StackBackward>)
torch.Size([3, 5, 2])


In [ ]:
# 1) library 불러오기
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn as nn

import torch.nn.init

# 2) device 설정하기
device = "cuda" if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cuda':
    torch.manual_seed_all(777)
    
# 3) hyper parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

# 4-1) dataset 가져오기
mnist_train = dsets.MNIST(root = "MNIST_data/", train= True, transform= transforms.ToTensor(), download= True)
mnist_test = dsets.MNIST(root = "MNIST_data/", train= False, transform= transforms.ToTensor(), download= True)

# 4-2) loader 만들기
from torch.utils.data import DataLoader
train_loader = DataLoader(dataset= mnist_train, batch_size= batch_size, shuffle= True, drop_last= True)
test_loader = DataLoader(dataset= mnist_test, batch_size= batch_size, shuffle= False, drop_last= False)

# 5) model 만들기

class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.convol = nn.Sequential(
            # layer1
            nn.Conv2d(1,32,kernel_size=3, stride=1, padding=1),
            nn.MaxPool2d(2),
            nn.ReLU(),
            
            #layer2
            nn.Conv2d(32,64,kernel_size=3, stride=1, padding=1),
            nn.MaxPool2d(2),
            nn.ReLU(),
            
            #layer3
            nn.Conv2d(64,128,kernel_size=3, stride=1, padding=1),
            nn.MaxPool2d(2),
            nn.ReLU(),
            
            
            
            # Outputs: 1*128*3*3
        )
        #layer3
        self.fc1 = nn.Linear(3*3*128, 625, bias = True)
        self.ReLu = nn.ReLU()
        self.fc2 = nn.Linear(625, 10, bias = True)
        
        nn.init.xavier_uniform_(self.fc1.weight)
        nn.init.xavier_uniform_(self.fc2.weight)
        
        
        
        
    def forward(self,x):
        y_ = self.convol(x)
        y_ = y_.view(y_.size(0),-1)
        y_ = self.fc1(y_)
        y_ = self.ReLu(y_)
        y_ = self.fc2(y_)
        
        return y_
    
model = CNN().to(device)

# 6) Loss & Optimizer
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)


# 7) training 
model.train()
total_batch = len(train_loader)
for epoch in range(training_epochs):
    avg_cost = 0
    for X,Y in train_loader:
        X = X.to(device)
        Y = Y.to(device)
        hypothesis = model(X)
        cost =criterion(hypothesis, Y)
        
        #backward
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
        
        avg_cost += cost.item() / total_batch
        
    print("[Epoch:{:04d}] cost = {:.4f}".format(epoch+1, avg_cost))
    #loss_tracker(loss_plt, torch.Tensor([avg_cost]), torch.Tensor([epoch]))
print("Learning Finished!")

#8) Test
with torch.no_grad():
    model.eval()
    X_test = mnist_test.test_data.view(len(mnist_test),1,28,28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)
    
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    acc = correct_prediction.float().mean()
    print("Accuracy:", acc.item())
    
